In [1]:
import polars as pl
import pandas as pd
from xgboost import XGBRegressor, XGBRFRegressor, plot_importance
import lightgbm as lgb
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.ensemble import VotingRegressor
import pickle
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

today= datetime.now()

In [73]:
df=pl.read_parquet("C:\\Users\\smishra14\\OneDrive - Stryker\\data\\Endoscopy.parquet")
df=df[['CatalogNumber','SALES_DATE','`Act Orders Rev','`Fcst Stat Prelim Rev','L2 Stat Final Rev']]
df1=df.filter(pl.col('SALES_DATE')<today-relativedelta(months=1))
df1=df1.group_by(['CatalogNumber','SALES_DATE']).sum()
df1=df1.sort('SALES_DATE',descending=False)
df1=df1.with_columns(pl.col('CatalogNumber').str.replace_all('/','-'))
cc=df1.filter(pl.col('SALES_DATE')>today-relativedelta(months=13)).group_by('CatalogNumber').sum().sort('`Act Orders Rev',descending=True)
cc=cc.filter(pl.col('`Act Orders Rev')>12)

def preprocess(df):
    df=df.with_columns(month=pl.col('SALES_DATE').dt.month())
    df=df.with_columns(days=(pl.col('SALES_DATE')-datetime(year=2021,month=1,day=1)).dt.total_days())
    df=df.with_columns(lag12=pl.col('`Act Orders Rev').shift(12).over('CatalogNumber'))
    df=df.with_columns(pl.when(pl.col('lag12').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag12')).alias('lag12'))
    return df

def fpreprocess(mon):
    fdf=pl.DataFrame({'SALES_DATE':pl.date_range(today.replace(day=1),today.replace(day=1)+relativedelta(months=mon-1), "1mo",eager=True)})
    fdf=fdf.join(cc[['CatalogNumber']],how='cross')
    fdf=fdf.with_columns(month=pl.col('SALES_DATE').dt.month())
    fdf=fdf.with_columns(days=(pl.col('SALES_DATE')-datetime(year=2021,month=1,day=1)).dt.total_days())
    df2=df1.with_columns(actwfc=pl.col('`Act Orders Rev'))
    fdf=pl.concat([preprocess(df2),fdf],how='diagonal_relaxed')
    fdf=fdf.with_columns(lag12=pl.col('`Act Orders Rev').shift(12).over('CatalogNumber'))
    fdf=fdf.with_columns(pl.when(pl.col('lag12').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag12')).alias('lag12'))
    return fdf

def fcpreprocess1(df,mon,cat):
    fdf=pl.DataFrame({'CatalogNumber':[cat]*mon,'SALES_DATE':pl.date_range(df['SALES_DATE'].max()+relativedelta(months=1),df['SALES_DATE'].max()+relativedelta(months=mon), "1mo",eager=True)})
    df=df.with_columns(actwfc=pl.col('`Act Orders Rev'))
    #fdf=fdf.with_columns(pred='')
    fdf=fdf.with_columns(month=pl.col('SALES_DATE').dt.month())
    fdf=fdf.with_columns(days=(pl.col('SALES_DATE')-datetime(year=2021,month=1,day=1)).dt.total_days())
    fdf=pl.concat([df,fdf],how='diagonal_relaxed')
    fdf=fdf.with_columns(lag12=pl.col('actwfc').shift(12).over('CatalogNumber'))
    fdf=fdf.with_columns(pl.when(pl.col('lag12').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag12')).alias('lag12'))
    return fdf

def fcpreprocess(df,mon,cat,pred):
    #fdf=pl.DataFrame({'SALES_DATE':pl.date_range(today.replace(day=1),today.replace(day=1)+relativedelta(months=mon-1), "1mo",eager=True)})
    df3=df.filter(pl.col('SALES_DATE')>df['SALES_DATE'].max()-relativedelta(months=mon)).with_columns(actwfc=pred,pred=pred)
    df=pl.concat([df.filter(pl.col('SALES_DATE')<=df['SALES_DATE'].max()-relativedelta(months=mon)),df3],how='diagonal_relaxed')
    fdf=pl.DataFrame({'CatalogNumber':[cat]*mon,'SALES_DATE':pl.date_range(df['SALES_DATE'].max()+relativedelta(months=1),df['SALES_DATE'].max()+relativedelta(months=mon), "1mo",eager=True)})
    fdf=fdf.with_columns(month=pl.col('SALES_DATE').dt.month())
    fdf=fdf.with_columns(days=(pl.col('SALES_DATE')-datetime(year=2021,month=1,day=1)).dt.total_days())
    #df=df.with_columns(actwfc=pl.col('`Act Orders Rev'))
    fdf=pl.concat([df,fdf],how='diagonal_relaxed')
    #prd=pl.DataFrame({'SALES_DATE':pl.date_range(today.replace(day=1),today.replace(day=1)+relativedelta(months=len(pred)-1), "1mo",eager=True),'XGBoost':pred})
    fdf=fdf.with_columns(lag12=pl.col('actwfc').shift(12).over('CatalogNumber'))
    fdf=fdf.with_columns(pl.when(pl.col('lag12').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag12')).alias('lag12'))
    return fdf


def pplot(df,pred,cat):
    pdf=df.filter(pl.col('CatalogNumber')==cat)
    prd=pl.DataFrame({'SALES_DATE':pl.date_range(today.replace(day=1),today.replace(day=1)+relativedelta(months=len(pred)-1), "1mo",eager=True),'XGBoost':pred})
    pdf=pdf.with_columns(pl.col('SALES_DATE').cast(pl.Date))
    pdf=pdf.join(prd,on='SALES_DATE', how="full", coalesce=True)
    pdf=pdf.sort('SALES_DATE',descending=False)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pdf['SALES_DATE'].to_list(),y=pdf['`Act Orders Rev'].to_list(),mode='lines',name='Act Orders',text=cat))
    fig.add_trace(go.Scatter(x=pdf['SALES_DATE'].to_list(),y=pdf['XGBoost'].to_list(),mode='lines',name='XGBoost'))
    return fig

def pplot1(df,cat):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['SALES_DATE'].to_list(),y=df['`Act Orders Rev'].to_list(),mode='lines',name='Act Orders',text=cat))
    fig.add_trace(go.Scatter(x=df['SALES_DATE'].to_list(),y=df['pred'].to_list(),mode='lines',name='XGBoost'))
    return fig

Tree based models cannot capture trends outside training data
https://medium.com/@simon.peter.mueller/overcoming-the-limitations-of-tree-based-models-in-time-series-forecasting-c2c5bd71a8f1

Hence below combined both tree based XGBoost and Linear booster 

In [69]:
pp=preprocess(df1)
for i in cc['CatalogNumber'][:1000]:
    p1=pp.filter(pl.col('CatalogNumber')==i).to_pandas()
    mod1=XGBRegressor()
    mod2=XGBRegressor(booster='gblinear')
    #mod2=LGBRegressor(linear_tree= True)
    pipe=VotingRegressor([('xgbm',mod1),('lgbm',mod2)])
    pipe.fit(X=p1[['month','days','lag12']],y=p1['`Act Orders Rev'])
    with open(f'models/{i}.pkl','wb') as f:
        pickle.dump(pipe,f)

In [72]:
fin_df=pl.DataFrame()
for i in cc['CatalogNumber'][:1000]:
    df2=df1.filter(pl.col('CatalogNumber')==i)
    df2=fcpreprocess1(df2,12,i)
    with open(f'models/{i}.pkl','rb') as f:
        pr=pickle.load(f)
        md=df1['SALES_DATE'].max()
        for t in range(5):
            pred=pr.predict(df2.filter(pl.col('SALES_DATE')>md)[['month','days','lag12']].to_pandas())
            md=df2['SALES_DATE'].max()
            df2=fcpreprocess(df2,12,i,pred)
    df2=df2.filter(pl.col('SALES_DATE')<=df2['SALES_DATE'].max()-relativedelta(months=12))
    fin_df=pl.concat([fin_df,df2])
fin_df.write_csv('forecast.csv')

In [74]:
df.write_csv('demantrastat.csv')

In [ ]:
page = []
for i in cc['CatalogNumber'][:30]:
    df2=df1.filter(pl.col('CatalogNumber')==i)
    df2=fcpreprocess1(df2,12,i)
    with open(f'models/{i}.pkl','rb') as f:
        pr=pickle.load(f)
        md=df1['SALES_DATE'].max()
        for t in range(5):
            pred=pr.predict(df2.filter(pl.col('SALES_DATE')>md)[['month','days','lag12']].to_pandas())
            md=df2['SALES_DATE'].max()
            df2=fcpreprocess(df2,12,i,pred)
    page.append(pplot1(df2,i))
rows = math.ceil(len(page) / 2)
fig1 = make_subplots(rows=rows, cols=2,subplot_titles=[i for i in cc['CatalogNumber'][:20]])
for i, chart in enumerate(page):
    fig1.add_traces([chart.data[0],chart.data[1]],rows=math.floor((i+2)/2), cols=i%2+1)
fig1.update_layout(height=2900,showlegend=False)
fig1.show()

## Without Lag

In [ ]:
fdf=pl.DataFrame({'SALES_DATE':pl.date_range(today.replace(day=1),today.replace(day=1)+relativedelta(months=60), "1mo",eager=True)})
fp=preprocess(fdf)
page = []
for i in cc['CatalogNumber'][:30]:
    with open(f'models/{i}.pkl','rb') as f:
        pr=pickle.load(f)
    pred=pr.predict(fp[['month','days']].to_pandas())
    page.append(pplot(df1,pred,i))
rows = math.ceil(len(page) / 2)
fig1 = make_subplots(rows=rows, cols=2,subplot_titles=[i for i in cc['CatalogNumber'][:20]])
for i, chart in enumerate(page):
    fig1.add_traces([chart.data[0],chart.data[1]],rows=math.floor((i+2)/2), cols=i%2+1)
fig1.update_layout(height=2900,showlegend=False)
fig1.show()

In [ ]:
pp=preprocess(df1)
for i in cc['CatalogNumber'][:40]:
    p1=pp.filter(pl.col('CatalogNumber')==i).to_pandas()
    mod1=XGBRegressor()
    mod2=XGBRegressor(booster='gblinear')
    #mod2=LGBRegressor(linear_tree= True)
    pipe=VotingRegressor([('xgbm',mod1),('lgbm',mod2)])
    pipe.fit(X=p1[['month','days']],y=p1['`Act Orders Rev'])
    with open(f'models/{i}.pkl','wb') as f:
        pickle.dump(pipe,f)

In [ ]:
fdf=pl.DataFrame({'SALES_DATE':pl.date_range(today.replace(day=1),today.replace(day=1)+relativedelta(months=60), "1mo",eager=True)})
fp=preprocess(fdf)
page = []
for i in cc['CatalogNumber'][:30]:
    with open(f'models/{i}.pkl','rb') as f:
        pr=pickle.load(f)
    pred=pr.predict(fp[['month','days']].to_pandas())
    page.append(pplot(df1,pred,i))
rows = math.ceil(len(page) / 2)
fig1 = make_subplots(rows=rows, cols=2,subplot_titles=[i for i in cc['CatalogNumber'][:20]])
for i, chart in enumerate(page):
    fig1.add_traces([chart.data[0],chart.data[1]],rows=math.floor((i+2)/2), cols=i%2+1)
fig1.update_layout(height=2900,showlegend=False)
fig1.show()

In [ ]:
def preprocess(df):
    df=df.with_columns(lag3=pl.col('`Act Orders Rev').shift(3).over('CatalogNumber'))
    df=df.with_columns(pl.when(pl.col('lag3').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag3')).alias('lag3'))
    df=df.with_columns(lag4=pl.col('`Act Orders Rev').shift(4).over('CatalogNumber'))
    df=df.with_columns(pl.when(pl.col('lag4').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag4')).alias('lag4'))
    df=df.with_columns(lag5=pl.col('`Act Orders Rev').shift(5).over('CatalogNumber'))
    df=df.with_columns(pl.when(pl.col('lag5').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag5')).alias('lag5'))
    df=df.with_columns(lag6=pl.col('`Act Orders Rev').shift(6).over('CatalogNumber'))
    df=df.with_columns(pl.when(pl.col('lag6').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag6')).alias('lag6'))
    df=df.with_columns(lag12=pl.col('`Act Orders Rev').shift(12).over('CatalogNumber'))
    df=df.with_columns(pl.when(pl.col('lag12').is_null()).then(pl.col('`Act Orders Rev')).otherwise(pl.col('lag12')).alias('lag12'))
    df=df.with_columns(month=pl.col('SALES_DATE').dt.month())
    return df

In [3]:
import polars as pl
df=pl.read_parquet('../data/Irrigation.parquet')
df=df.with_columns(pl.col('`Act Orders Rev','`Fcst Stat Prelim Rev').cast(pl.Float32))
df.write_parquet('../data/Irrigation.parquet')